# Notebook 01 — Dataset Construction from MIMIC-IV / MIMIC-IV-Note

Mục tiêu: xây dựng một tập dữ liệu gọn để huấn luyện mô hình văn bản → đa nhãn (ICD-block và nhóm xét nghiệm giai đoạn sớm), với khả năng giới hạn số dòng đọc ở mọi bước nhằm phục vụ demo nhanh và tiết kiệm tài nguyên.

Quy ước trình bày cho mỗi bước:
1) Mục đích của bước và lý do cần thiết.
2) Trường dữ liệu được sử dụng và ý nghĩa của từng trường; nếu nối giữa các bảng, chỉ rõ khóa nối và ý nghĩa của khóa.
3) Thực hiện xử lý dữ liệu.
4) Diễn giải ý nghĩa các trường trong kết quả hiển thị.

## 0) Cấu hình & nguyên tắc đọc dữ liệu (không dùng `nrows`)

**Quan trọng:** Không tự động giới hạn số dòng bằng `nrows`. Thay vào đó:
- Chọn **COHORT** ngay ở bước setup bằng biến `N_HADM` (số lần nhập viện muốn lấy).
- Ở các bước sau, **đọc theo `chunksize`** và **lọc theo `COHORT_HADM`** để đảm bảo khóa nhất quán giữa admissions/ICD/Lab/Notes.

**Tham số chính trong bước này:**
- `N_HADM` *(int | None)*: số lần nhập viện trong cohort (đặt `None` để dùng toàn bộ).
- `SEED` *(int)*: hạt giống ngẫu nhiên khi cần chọn ngẫu nhiên.
- `BALANCE_BY_SUBJECT` *(bool)*: gợi ý cân bằng theo `subject_id` khi chọn cohort.
- `CHUNKSIZE_DEFAULT` *(int)*: kích thước chunk chuẩn khi đọc các bảng lớn.
- `TOP_LABS` *(int)*: số lab phổ biến (0–6h) dùng làm vocab.

In [10]:
# ======================================================
# BƯỚC 1 — CẤU HÌNH DỮ LIỆU VÀ THAM SỐ TIỀN XỬ LÝ
# ======================================================

from pathlib import Path

# ====== Tham số cohort & đọc file ======
N_HADM = 10000              # vd: 300_000 hoặc None để dùng toàn bộ admissions
SEED = 42                  # reproducibility
BALANCE_BY_SUBJECT = True
CHUNKSIZE_DEFAULT = 500_000  # đọc file lớn theo chunk

# ====== Giới hạn vocab / feature ======
TOP_LABS = 50              # số lượng lab phổ biến (toàn kỳ)
TOP_ICD = 50               # số lượng ICD-block phổ biến (toàn kỳ)
TOP_PROCS = 50             # số lượng thủ thuật phổ biến (toàn kỳ)

# ====== Dò đường dẫn dữ liệu ======
CANDIDATES = [Path("data"), Path("../data"), Path("../../data")]
DATA_ROOT = None
for cand in CANDIDATES:
    if (cand / "mimiciv").exists() and (cand / "mimic-iv-note").exists():
        DATA_ROOT = cand
        break
if DATA_ROOT is None:
    raise FileNotFoundError(
        "❌ Không tìm thấy thư mục 'data' chứa 'mimiciv' và 'mimic-iv-note'. "
        "Hãy điều chỉnh CANDIDATES hoặc thiết lập DATA_ROOT thủ công."
    )

# ====== Khai báo đường dẫn ======
HOSP_DIR = DATA_ROOT / "mimiciv" / "3.1" / "hosp"
ICU_DIR  = DATA_ROOT / "mimiciv" / "3.1" / "icu"
NOTE_DIR = DATA_ROOT / "mimic-iv-note" / "2.2" / "note"
PROC_DIR = DATA_ROOT / "proc"
PROC_DIR.mkdir(parents=True, exist_ok=True)

# ====== In thông tin cấu hình ======
print("DATA_ROOT:", DATA_ROOT.resolve())
print("HOSP_DIR:", HOSP_DIR.resolve())
print("NOTE_DIR:", NOTE_DIR.resolve())
print("PROC_DIR:", PROC_DIR.resolve())
print({
    "N_HADM": N_HADM,
    "SEED": SEED,
    "BALANCE_BY_SUBJECT": BALANCE_BY_SUBJECT,
    "CHUNKSIZE_DEFAULT": CHUNKSIZE_DEFAULT,
    "TOP_LABS": TOP_LABS,
    "TOP_ICD": TOP_ICD,
    "TOP_PROCS": TOP_PROCS,
})


DATA_ROOT: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data
HOSP_DIR: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimiciv/3.1/hosp
NOTE_DIR: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/mimic-iv-note/2.2/note
PROC_DIR: /Users/lehoangkhang/Tài liệu/revita-sympdiag/data/proc
{'N_HADM': 10000, 'SEED': 42, 'BALANCE_BY_SUBJECT': True, 'CHUNKSIZE_DEFAULT': 500000, 'TOP_LABS': 50, 'TOP_ICD': 50, 'TOP_PROCS': 50}


### Tiện ích đọc dữ liệu & xử lý cơ bản
- `read_csv_chunks(path, usecols=None, chunksize=CHUNKSIZE_DEFAULT)`: **generator** đọc theo chunk, KHÔNG dùng `nrows`.
- `normalize_text(s)`: chuẩn hóa khoảng trắng, cắt 4,000 ký tự.
- `icd_to_block(icd_code)`: rút gọn mã ICD về block 3 ký tự (bỏ dấu chấm).

**Kết quả mong đợi:**
- In xác nhận đã nạp tiện ích.

**Ý nghĩa trường trả về:**
- `read_csv_chunks(...) → Iterator[DataFrame]`: dùng trong vòng lặp để lọc theo `COHORT_HADM`.
- `normalize_text(s) → str`: văn bản sạch, ổn định tokenize.
- `icd_to_block(code) → str|nan`: nhãn ICD rút gọn, giảm chiều không gian nhãn.

In [11]:
import pandas as pd
import numpy as np
from typing import Iterator, Optional, List

# ======================================================
# BƯỚC 2 — HÀM TIỆN ÍCH CHUNG CHO TIỀN XỬ LÝ
# ======================================================

def read_csv_chunks(path, usecols: Optional[List[str]] = None, chunksize: int = CHUNKSIZE_DEFAULT) -> Iterator[pd.DataFrame]:
    """
    Đọc file .csv.gz theo từng khối (chunk) để tiết kiệm RAM.
    KHÔNG dùng nrows.
    """
    return pd.read_csv(
        path,
        usecols=usecols,
        chunksize=chunksize,
        compression="gzip"
    )

def normalize_text(s):
    """Chuẩn hóa text: loại khoảng trắng dư, cắt tối đa 4000 ký tự."""
    if pd.isna(s):
        return ""
    return " ".join(str(s).split())[:4000]

def icd_to_block(icd_code: str) -> str:
    """Chuyển mã ICD thành block 3 ký tự (VD: 'E119' → 'E11')."""
    if pd.isna(icd_code):
        return np.nan
    s = str(icd_code).replace('.', '').strip()
    return s[:3] if len(s) >= 3 else s

def proc_to_block(proc_code: str) -> str:
    """
    Chuẩn hóa mã thủ thuật (procedure) ICD:
    - Loại bỏ dấu chấm.
    - Với ICD-9: cắt 3 ký tự đầu (ví dụ '88.72' → '887').
    - Với ICD-10: lấy 4 ký tự đầu (ví dụ '0UT9FZZ' → '0UT9').
    """
    if pd.isna(proc_code):
        return np.nan
    s = str(proc_code).replace('.', '').strip().upper()
    return s[:3] if s[0].isdigit() else s[:4]

pd.set_option("display.max_colwidth", 180)
pd.set_option("display.width", 140)

print("✅ Đã nạp tiện ích: read_csv_chunks, normalize_text, icd_to_block, proc_to_block")

✅ Đã nạp tiện ích: read_csv_chunks, normalize_text, icd_to_block, proc_to_block


## 1) Admissions — mốc thời gian nhập viện & khởi tạo COHORT

**Mục đích**
1) Lấy mốc `admittime` cho từng `hadm_id` để tính các cửa sổ sớm: xét nghiệm **0–6h**, ghi chú **0–12h**.
2) **Tạo COHORT_HADM** theo cấu hình ở bước setup (`N_HADM`, `BALANCE_BY_SUBJECT`, `SEED`). Các bảng sau (ICD/Lab/Note) sẽ **lọc theo cohort** khi đọc theo `chunksize`.

**Tham số ảnh hưởng** (đã khai báo ở bước setup):
- `N_HADM` *(int | None)*: số lần nhập viện cần lấy vào cohort. `None` = dùng toàn bộ `admissions`.
- `BALANCE_BY_SUBJECT` *(bool)*: nếu `True`, duyệt theo `subject_id` để hạn chế thiên lệch (nhiều HADM từ cùng một bệnh nhân).
- `SEED` *(int)*: hạt giống ngẫu nhiên khi cần lấy mẫu.

**Kết quả mong đợi**
- `COHORT_HADM` *(set[int])*: tập `hadm_id` cố định dùng xuyên suốt pipeline.
- `admissions_idx` *(Series indexed by `hadm_id`)*: tra cứu nhanh `admittime` phục vụ join thời gian cho Lab/Note.

**Ý nghĩa trường dữ liệu trong kết quả**
- `hadm_id`: mã lần nhập viện — khóa nối giữa các bảng.
- `admittime`: mốc thời gian nhập viện — căn cứ cắt cửa sổ 0–6h (Lab) và 0–12h (Note).
- `subject_id`: mã bệnh nhân (chỉ dùng khi cân bằng cohort theo bệnh nhân).

In [12]:
# ======================================================
# BƯỚC 1 — ĐỌC ADMISSIONS & TẠO COHORT_HADM (CHỈ NHỮNG CA CÓ GHI CHÚ XUẤT VIỆN)
# ======================================================

import numpy as np
import pandas as pd

adm_path = HOSP_DIR / "admissions.csv.gz"
discharge_path = NOTE_DIR / "discharge.csv.gz"

# ------------------------------------------------------
# Đọc danh sách admissions (toàn bộ)
# ------------------------------------------------------
usecols = ["hadm_id", "subject_id", "admittime"]
admissions = pd.read_csv(adm_path, usecols=usecols, parse_dates=["admittime"], compression="gzip")

# Làm sạch dữ liệu cơ bản
admissions = (
    admissions
    .dropna(subset=["hadm_id", "admittime"])
    .drop_duplicates(subset=["hadm_id"])
)
print(f"📘 Tổng số lượt nhập viện ban đầu: {len(admissions):,}")

# ------------------------------------------------------
# Đọc ghi chú xuất viện để xác định các HADM có text
# ------------------------------------------------------
usecols = ["hadm_id", "charttime", "text"]
print("🔍 Đang đọc ghi chú xuất viện...")
dis_rows = []

for chunk in read_csv_chunks(discharge_path, usecols=usecols):
    chunk = chunk.dropna(subset=["hadm_id", "text"]).copy()
    if chunk.empty:
        continue
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    chunk["text"] = chunk["text"].map(normalize_text)
    dis_rows.append(chunk[["hadm_id", "charttime", "text"]])

if dis_rows:
    discharge_df = pd.concat(dis_rows, ignore_index=True)
    # Chọn note sớm nhất (tránh trùng ghi chú discharge)
    discharge_df = (
        discharge_df.sort_values(["hadm_id", "charttime"], ascending=[True, True])
        .drop_duplicates(subset=["hadm_id"], keep="first")
    )
else:
    discharge_df = pd.DataFrame(columns=["hadm_id", "text"])

# ------------------------------------------------------
# Giữ lại cohort admissions có ghi chú xuất viện
# ------------------------------------------------------
HADM_WITH_NOTE = set(discharge_df["hadm_id"].astype(int))
admissions = admissions[admissions["hadm_id"].isin(HADM_WITH_NOTE)]

# ------------------------------------------------------
# Tạo COHORT_HADM (lấy toàn bộ hoặc lấy mẫu nếu giới hạn N_HADM)
# ------------------------------------------------------
if (N_HADM is None) or (N_HADM >= len(admissions)):
    COHORT_HADM = set(admissions["hadm_id"].astype(int))
else:
    rng = np.random.default_rng(SEED)
    if BALANCE_BY_SUBJECT:
        subjects = admissions["subject_id"].dropna().astype(int).unique()
        rng.shuffle(subjects)
        picked = []
        for sid in subjects:
            rows = admissions.loc[admissions["subject_id"] == sid, "hadm_id"].astype(int).tolist()
            picked.extend(rows)
            if len(picked) >= N_HADM:
                break
        COHORT_HADM = set(picked[:N_HADM])
    else:
        hadms = admissions["hadm_id"].astype(int).values
        sel = rng.choice(hadms, size=N_HADM, replace=False)
        COHORT_HADM = set(int(x) for x in sel)

# ------------------------------------------------------
# Tạo index thời gian nhập viện (để join với các bảng khác)
# ------------------------------------------------------
admissions_cohort = admissions[admissions["hadm_id"].isin(COHORT_HADM)].copy()
admissions_idx = admissions_cohort.set_index("hadm_id")["admittime"].sort_index()

print(f"✅ Số lượt nhập viện có ghi chú xuất viện: {len(COHORT_HADM):,}")
display(admissions_idx.to_frame().head())

print("Giải thích kết quả:")
print("- hadm_id (index): mã lần nhập viện trong cohort (có ghi chú xuất viện).")
print("- admittime: thời điểm nhập viện để tham chiếu các bảng khác (ICD, thủ thuật, xét nghiệm, v.v.).")

📘 Tổng số lượt nhập viện ban đầu: 546,028
🔍 Đang đọc ghi chú xuất viện...
✅ Số lượt nhập viện có ghi chú xuất viện: 10,000


,admittime
hadm_id,
20000815,2162-03-08 19:52:00
20004116,2202-06-21 14:15:00
20004829,2172-05-25 16:52:00
20004866,2126-09-22 16:12:00
20006409,2126-10-28 00:28:00


Giải thích kết quả:
- hadm_id (index): mã lần nhập viện trong cohort (có ghi chú xuất viện).
- admittime: thời điểm nhập viện để tham chiếu các bảng khác (ICD, thủ thuật, xét nghiệm, v.v.).


## 2) ICD-block cho từng lần nhập viện (lọc theo COHORT, đọc theo `chunksize`)

**Mục đích**  
- Rút gọn các mã ICD chi tiết thành **ICD-block** (3 ký tự đầu) để giảm số nhãn, vẫn giữ thông tin nhóm bệnh chính.

**Dữ liệu đầu vào & ràng buộc**  
- Dựa trên **COHORT_HADM** đã chọn ở Bước 1.
- Đọc `diagnoses_icd.csv.gz` theo **chunksize** và **lọc theo `hadm_id ∈ COHORT_HADM`** để đảm bảo nhất quán khóa.

**Trường dùng**  
- `subject_id`: mã bệnh nhân  
- `hadm_id`: mã lần nhập viện (khóa nối chính)  
- `icd_code`: mã ICD gốc → chuyển về **ICD-block**

**Kết quả mong đợi**  
- Bảng `icd_df` gồm: `subject_id`, `hadm_id`, `icd_blocks` (danh sách ICD-block **duy nhất** của ca đó).

**Ý nghĩa trường trong kết quả**  
- `icd_blocks`: danh sách các ICD-block (3 ký tự) đại diện nhóm bệnh chính cho mỗi lần nhập viện.

**Lưu ý**  
- Không còn dùng `nrows`; thay vào đó **đọc full theo `chunksize`** và lọc theo **COHORT** ngay trong vòng lặp.

In [13]:
# --- Bước 2: Đọc ICD theo chunksize & lọc theo COHORT_HADM, tạo icd_df ---
import pandas as pd

diag_path = HOSP_DIR / "diagnoses_icd.csv.gz"

icd_rows = []
for chunk in read_csv_chunks(
    diag_path,
    usecols=["subject_id", "hadm_id", "icd_code"],
    chunksize=CHUNKSIZE_DEFAULT,
):
    # làm sạch cơ bản và lọc theo cohort
    chunk = chunk.dropna(subset=["hadm_id", "icd_code"]).copy()
    if chunk.empty:
        continue
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    chunk = chunk[chunk["hadm_id"].isin(COHORT_HADM)]
    if chunk.empty:
        continue
    # chuyển icd_code -> block
    chunk["block"] = chunk["icd_code"].map(icd_to_block)
    icd_rows.append(chunk[["subject_id", "hadm_id", "block"]])

if icd_rows:
    diag_df = pd.concat(icd_rows, ignore_index=True)
else:
    diag_df = pd.DataFrame(columns=["subject_id", "hadm_id", "block"])  # rỗng an toàn

icd_df = (
    diag_df
    .groupby(["subject_id", "hadm_id"]) ["block"]
    .apply(lambda x: sorted({b for b in x if pd.notna(b)}))
    .reset_index()
    .rename(columns={"block": "icd_blocks"})
)

# sanity checks
hadm_icd_n = icd_df["hadm_id"].nunique()
print("ICD — số hadm có nhãn:", hadm_icd_n, "/", len(COHORT_HADM))
display(icd_df.head())

print("Giải thích kết quả:")
print("- subject_id: mã bệnh nhân.")
print("- hadm_id: mã lần nhập viện (khóa nối chính với Lab/Note).")
print("- icd_blocks: danh sách ICD-block (3 ký tự) — nhóm bệnh chính.")

ICD — số hadm có nhãn: 9996 / 10000


,subject_id,hadm_id,icd_blocks
0,10000935,21738619,"[266, 272, 276, 288, 311, 427, 715, 721, 780, 786, 787, 793, V15, V45, V58]"
1,10000935,25849114,"[153, 197, 266, 272, 276, 285, 286, 288, 300, 311, 348, 427, 578, 715, 721, 729, 788, V15, V16, V45, V49, V88]"
2,10000935,26381316,"[197, 199, 266, 272, 276, 288, 311, 338, 564, 715, 721, 724, 780, 782, 783, 786, 787, 789, V10, V15, V45, V85]"
3,10000935,29541074,"[266, 272, 278, 311, 560, 788, 998, E87, V10, V88]"
4,10002430,24513842,"[E78, E87, I10, I25, I27, I28, I48, I50, J44, K21, N40, Z79, Z87, Z90, Z98]"


Giải thích kết quả:
- subject_id: mã bệnh nhân.
- hadm_id: mã lần nhập viện (khóa nối chính với Lab/Note).
- icd_blocks: danh sách ICD-block (3 ký tự) — nhóm bệnh chính.


### 2.5) Demographics: giới tính & tuổi tại thời điểm nhập viện

**Mục đích**  
- Tính **tuổi tại nhập viện** và lấy **giới tính** để dùng làm đặc trưng tabular (Bước train).

**Công thức**  
- `age_at_admit = anchor_age + (admit_year - anchor_year)`  
- Sau đó **clip** về `[0, 120]`, **round** và lưu kiểu số nguyên `Int64` (nullable).

**Dữ liệu & ràng buộc**  
- Lấy `subject_id` ↔ `hadm_id` từ `icd_df` (đã lọc theo cohort).
- Năm nhập viện `admit_year` lấy từ `admissions_idx` (Bước 1).  
- Đọc `patients.csv.gz` đầy đủ (file vừa phải) để tính tuổi & lấy giới tính.

**Kết quả mong đợi**  
- Bảng `demo_df` gồm: `hadm_id`, `subject_id`, `gender` (M/F/U), `age_at_admit` (0–120, Int64).

In [14]:
# --- Bước 2.5: Tính demographics (gender, age_at_admit) ---
pat_path = HOSP_DIR / "patients.csv.gz"

# Đọc demographics từ bảng patients
patients = pd.read_csv(
    pat_path,
    usecols=["subject_id", "gender", "anchor_age", "anchor_year"],
    compression="gzip",
)
patients["gender"] = patients["gender"].astype(str).str.upper().str[0]  # 'M'/'F'/'U'

# Lấy subject_id từ các hadm có note xuất viện (COHORT_HADM)
# Ở bước 1 bạn đã có admissions_cohort chứa hadm_id, subject_id
hadm_subject = admissions_cohort[["hadm_id", "subject_id"]].drop_duplicates()

# Lấy năm nhập viện từ admissions_idx (Series indexed by hadm_id)
adm_year = admissions_idx.to_frame(name="admittime").reset_index()
adm_year["admit_year"] = adm_year["admittime"].dt.year

# Ghép để tính tuổi tại thời điểm nhập viện
age_df = (
    hadm_subject
    .merge(patients, on="subject_id", how="left")
    .merge(adm_year[["hadm_id", "admit_year"]], on="hadm_id", how="left")
)
age_df["age_at_admit"] = age_df["anchor_age"] + (age_df["admit_year"] - age_df["anchor_year"])
age_df["age_at_admit"] = (
    age_df["age_at_admit"].clip(lower=0, upper=120).round().astype("Int64")
)

demo_df = age_df[["hadm_id", "subject_id", "gender", "age_at_admit"]].copy()

# Đảm bảo chỉ giữ đúng các lượt nhập viện có note xuất viện
demo_df = demo_df[demo_df["hadm_id"].isin(COHORT_HADM)]

# sanity checks
print("Demo — số hadm có demographics:", demo_df["hadm_id"].nunique(), "/", len(COHORT_HADM))
display(demo_df.head())

print("Giải thích kết quả:")
print("- Chỉ tính cho các lượt nhập viện có note xuất viện.")
print("- gender: M/F/U.")
print("- age_at_admit: tuổi tại thời điểm nhập viện (0–120, kiểu Int64).")


Demo — số hadm có demographics: 10000 / 10000


,hadm_id,subject_id,gender,age_at_admit
0,21738619,10000935,F,57
1,25849114,10000935,F,57
2,26381316,10000935,F,57
3,29541074,10000935,F,53
4,24513842,10002430,M,86


Giải thích kết quả:
- Chỉ tính cho các lượt nhập viện có note xuất viện.
- gender: M/F/U.
- age_at_admit: tuổi tại thời điểm nhập viện (0–120, kiểu Int64).


In [24]:
# ======================================================
# BƯỚC 3 — ĐỌC THỦ THUẬT (PROCEDURES ICD) & TẠO proc_df + CÁC FILE THỐNG KÊ
# ======================================================

import pandas as pd
from collections import Counter

proc_path = HOSP_DIR / "procedures_icd.csv.gz"

proc_rows = []
for chunk in read_csv_chunks(
    proc_path,
    usecols=["subject_id", "hadm_id", "icd_code"],
    chunksize=CHUNKSIZE_DEFAULT,
):
    # Làm sạch cơ bản và lọc theo cohort
    chunk = chunk.dropna(subset=["hadm_id", "icd_code"]).copy()
    if chunk.empty:
        continue
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    chunk = chunk[chunk["hadm_id"].isin(COHORT_HADM)]
    if chunk.empty:
        continue

    # Chuẩn hóa và tách block
    chunk["icd_code"] = chunk["icd_code"].astype(str).str.replace(".", "", regex=False)
    chunk["block"] = chunk["icd_code"].str[:3]
    proc_rows.append(chunk[["subject_id", "hadm_id", "block", "icd_code"]])

# ------------------------------------------------------
# HỢP NHẤT CÁC CHUNK
# ------------------------------------------------------
if proc_rows:
    proc_df_raw = pd.concat(proc_rows, ignore_index=True)
else:
    proc_df_raw = pd.DataFrame(columns=["subject_id", "hadm_id", "block", "icd_code"])

# ------------------------------------------------------
# 3.1 — GOM NHÓM CHO MỖI LẦN NHẬP VIỆN
# ------------------------------------------------------
proc_df = (
    proc_df_raw
    .groupby(["subject_id", "hadm_id"])["block"]
    .apply(lambda x: sorted({b for b in x if pd.notna(b)}))
    .reset_index()
    .rename(columns={"block": "proc_blocks"})
)

hadm_proc_n = proc_df["hadm_id"].nunique()
print("PROCEDURE — số hadm có thủ thuật:", hadm_proc_n, "/", len(COHORT_HADM))

# ------------------------------------------------------
# 3.2 — XUẤT TOP CÁC THỦ THUẬT (BLOCK)
# ------------------------------------------------------
cnt_proc = Counter(proc_df_raw["block"].dropna().astype(str))
top_proc_df = (
    pd.DataFrame(cnt_proc.items(), columns=["proc_block", "count"])
    .sort_values("count", ascending=False)
    .reset_index(drop=True)
)

TOP_PROCS = 100
out_top_proc = PROC_DIR / "top_procedures.csv"
top_proc_df.head(TOP_PROCS).to_csv(out_top_proc, index=False)
print(f"✅ Đã lưu danh sách Top {TOP_PROCS} thủ thuật:", out_top_proc)

# ------------------------------------------------------
# 3.3 — TẠO FILE PHÂN CẤP (BLOCK → MÃ CHI TIẾT)
# ------------------------------------------------------
freq_detail = (
    proc_df_raw
    .groupby(["block", "icd_code"])["hadm_id"]
    .nunique()
    .reset_index(name="n_hadm")
    .sort_values(["block", "n_hadm"], ascending=[True, False])
)

# Gắn mô tả nếu có dictionary
proc_dict_path = HOSP_DIR / "d_icd_procedures.csv.gz"
if proc_dict_path.exists():
    d_proc = pd.read_csv(proc_dict_path, usecols=["icd_code", "long_title"], compression="gzip")
    d_proc["icd_code"] = d_proc["icd_code"].astype(str).str.replace(".", "", regex=False)
    freq_detail = freq_detail.merge(d_proc, on="icd_code", how="left")

out_hierarchy = PROC_DIR / "top_procedure_hierarchy.csv"
freq_detail.to_csv(out_hierarchy, index=False)
print(f"✅ Đã lưu cấu trúc phân cấp thủ thuật:", out_hierarchy)

# ------------------------------------------------------
# 3.4 — KIỂM TRA NHANH
# ------------------------------------------------------
print("Tổng số thủ thuật (record-level):", len(proc_df_raw))
print("Tổng số mã thủ thuật khác nhau:", proc_df_raw["icd_code"].nunique())
display(top_proc_df.head(10))
display(freq_detail.head(10))
display(proc_df.head())

print("Giải thích:")
print("- proc_df: dùng cho merge ở bước 5.")
print("- top_procedures.csv: thống kê tần suất 3 ký tự đầu (block).")
print("- top_procedure_hierarchy.csv: chi tiết từng mã thủ thuật con trong từng block.")


PROCEDURE — số hadm có thủ thuật: 6019 / 10000
✅ Đã lưu danh sách Top 100 thủ thuật: ../data/proc/top_procedures.csv
✅ Đã lưu cấu trúc phân cấp thủ thuật: ../data/proc/top_procedure_hierarchy.csv
Tổng số thủ thuật (record-level): 18026
Tổng số mã thủ thuật khác nhau: 2913


,proc_block,count
0,389,1013
1,02H,487
2,3E0,483
3,5A1,465
4,004,371
5,0W9,339
6,885,328
7,967,315
8,884,295
9,372,280


,block,icd_code,n_hadm,long_title
0,001,0014,16,Injection or infusion of oxazolidinone class of antibiotics
1,001,0015,11,High-dose infusion interleukin-2 [IL-2]
2,001,00163J6,3,"Bypass Cerebral Ventricle to Peritoneal Cavity with Synthetic Substitute, Percutaneous Approach"
3,001,0017,3,Infusion of vasopressor agent
4,001,0011,1,Infusion of drotrecogin alfa (activated)
5,001,0012,1,Administration of inhaled nitric oxide
6,001,00163J4,1,"Bypass Cerebral Ventricle to Pleural Cavity with Synthetic Substitute, Percutaneous Approach"
7,002,0024,3,Intravascular imaging of coronary vessels
8,002,0023,1,Intravascular imaging of peripheral vessels
9,003,0039,2,Other computer assisted surgery


,subject_id,hadm_id,proc_blocks
0,10000935,25849114,[501]
1,10000935,29541074,"[456, 545]"
2,10002430,27218502,[0YU]
3,10003199,21858062,[793]
4,10003203,25146997,"[048, 793, 836]"


Giải thích:
- proc_df: dùng cho merge ở bước 5.
- top_procedures.csv: thống kê tần suất 3 ký tự đầu (block).
- top_procedure_hierarchy.csv: chi tiết từng mã thủ thuật con trong từng block.


## 3) Nhóm xét nghiệm trong 6 giờ đầu (lọc theo COHORT, đọc theo `chunksize`)

**Mục đích**
- Xây dựng nhãn về **các xét nghiệm được thực hiện sớm** (0–6 giờ đầu kể từ `admittime`) để huấn luyện mô hình gợi ý cận lâm sàng.

**Dữ liệu đầu vào & ràng buộc**
- `labevents.csv.gz`: `hadm_id`, `itemid`, `charttime`  
- `d_labitems.csv.gz`: `itemid` → `label`  
- `admissions_idx` (Bước 1): tra cứu `admittime` theo `hadm_id`  
- **Chỉ lấy bản ghi có `hadm_id ∈ COHORT_HADM`**. Đọc theo `chunksize` để tiết kiệm RAM.

**Xử lý (2 pass)**
1) **Đếm tần suất** `itemid` trong 0–6h để chọn `TOP_LABS` (vocab xét nghiệm sớm phổ biến).  
2) **Gán nhãn cho từng ca**: với mỗi `hadm_id`, lấy tập hợp `itemid` (thuộc vocab) diễn ra trong 0–6h.

**Kết quả mong đợi**
- `lab_vocab_df`: bảng vocab xét nghiệm sớm gồm `itemid`, `label`, `count` (tần suất).  
- `lab_items_by_hadm`: cho *mỗi* `hadm_id`, danh sách duy nhất các `itemid` (thuộc vocab) xuất hiện trong 0–6h.  
- (Tiện ích) `itemid_to_label`: ánh xạ `itemid → label` để hiển thị dễ đọc.

**Ý nghĩa trường trong kết quả**
- `itemid`: mã xét nghiệm.  
- `label`: tên xét nghiệm.  
- `count`: số lần xuất hiện trong cửa sổ 0–6h (trên toàn cohort).  
- `lab_items` (ở `lab_items_by_hadm`): danh sách *duy nhất* các `itemid` thuộc vocab của từng ca.

In [16]:
# --- Bước 4: Lấy xét nghiệm Top-N theo tần suất (toàn bộ thời gian nhập viện) ---
from collections import Counter
import pandas as pd

labs_path = HOSP_DIR / "labevents.csv.gz"
dlab_path = HOSP_DIR / "d_labitems.csv.gz"

# 4.1 Đọc dictionary lab: itemid -> label
dlab = pd.read_csv(dlab_path, usecols=["itemid", "label"], compression="gzip")
dlab["itemid"] = dlab["itemid"].astype(int)
dlab["label"] = dlab["label"].astype(str)

# 4.2 Pass 1 — Đếm tần suất itemid trong toàn bộ cohort
cnt = Counter()

for chunk in read_csv_chunks(
    labs_path,
    usecols=["hadm_id", "itemid"],
    chunksize=CHUNKSIZE_DEFAULT,
):
    chunk = chunk.dropna(subset=["hadm_id", "itemid"]).copy()
    if chunk.empty:
        continue
    chunk["hadm_id"] = chunk["hadm_id"].astype(int)
    chunk["itemid"]  = chunk["itemid"].astype(int)
    # Chỉ lấy các lượt nhập viện có note xuất viện
    chunk = chunk[chunk["hadm_id"].isin(COHORT_HADM)]
    if chunk.empty:
        continue
    cnt.update(chunk["itemid"].value_counts().to_dict())

# Tính tần suất
counts_df = (
    pd.DataFrame(list(cnt.items()), columns=["itemid", "count"])
    if cnt else pd.DataFrame(columns=["itemid", "count"])
)
counts_df = counts_df.sort_values(["count", "itemid"], ascending=[False, True])
top_items = counts_df.head(TOP_LABS)["itemid"].astype(int).tolist()

lab_vocab_df = (
    counts_df[counts_df["itemid"].isin(top_items)]
    .merge(dlab, on="itemid", how="left")
    [["itemid", "label", "count"]]
    .reset_index(drop=True)
)

# 4.3 Pass 2 — Gán danh sách xét nghiệm theo hadm_id (không giới hạn thời gian)
lab_rows = []

if len(top_items) > 0:
    for chunk in read_csv_chunks(
        labs_path,
        usecols=["hadm_id", "itemid"],
        chunksize=CHUNKSIZE_DEFAULT,
    ):
        chunk = chunk.dropna(subset=["hadm_id", "itemid"]).copy()
        if chunk.empty:
            continue
        chunk["hadm_id"] = chunk["hadm_id"].astype(int)
        chunk["itemid"]  = chunk["itemid"].astype(int)
        chunk = chunk[chunk["hadm_id"].isin(COHORT_HADM)]
        if chunk.empty:
            continue
        early_top = chunk[chunk["itemid"].isin(top_items)]
        if not early_top.empty:
            lab_rows.append(early_top[["hadm_id", "itemid"]])

if lab_rows:
    labs_concat = pd.concat(lab_rows, ignore_index=True)
    lab_items_by_hadm = (
        labs_concat
        .groupby("hadm_id")["itemid"]
        .apply(lambda s: sorted(set(s.tolist())))
        .reset_index()
        .rename(columns={"itemid": "lab_items"})
    )
else:
    lab_items_by_hadm = pd.DataFrame(columns=["hadm_id", "lab_items"])

# Ánh xạ itemid -> label (để hiển thị dễ hiểu hơn)
itemid_to_label = dict(zip(lab_vocab_df["itemid"], lab_vocab_df["label"]))

# Báo cáo nhanh
print("Số phần tử vocab xét nghiệm (TOP_N):", len(lab_vocab_df))
print("Số ca có ít nhất một xét nghiệm trong TOP_N:", lab_items_by_hadm["hadm_id"].nunique(), "/", len(COHORT_HADM))
display(lab_vocab_df.head(10))
display(lab_items_by_hadm.head(10))

# 4.4 Lưu danh sách Top Labs
out_top_labs = PROC_DIR / "top_lab_items.csv"
lab_vocab_df.to_csv(out_top_labs, index=False)
print("Đã lưu danh sách Top Labs:", out_top_labs)


Số phần tử vocab xét nghiệm (TOP_N): 50
Số ca có ít nhất một xét nghiệm trong TOP_N: 9395 / 10000


,itemid,label,count
0,50971,Potassium,57874
1,50983,Sodium,57319
2,50902,Chloride,56839
3,51221,Hematocrit,56608
4,50912,Creatinine,56355
5,51006,Urea Nitrogen,55954
6,50882,Bicarbonate,55406
7,50868,Anion Gap,55279
8,50931,Glucose,54769
9,51265,Platelet Count,53634


,hadm_id,lab_items
0,20000815,"[50861, 50863, 50868, 50878, 50882, 50885, 50902, 50912, 50931, 50954, 50971, 50983, 51006, 51221, 51222, 51248, 51249, 51250, 51265, 51277, 51279, 51301]"
1,20004829,"[50861, 50862, 50863, 50868, 50878, 50882, 50885, 50893, 50902, 50912, 50931, 50960, 50970, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51237, 51244, 51248, 51249, 51250, ..."
2,20004866,"[50861, 50863, 50868, 50878, 50882, 50885, 50893, 50902, 50912, 50931, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51248, 51249, 51250, 51265, 51277, 51279, 51301]"
3,20006409,"[50802, 50804, 50813, 50818, 50820, 50821, 50868, 50882, 50893, 50902, 50912, 50920, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51237, 51248, 51249, ..."
4,20007388,"[50868, 50882, 50902, 50912, 50920, 50931, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51244, 51248, 51249, 51250, 51254, 51256, 51265, 51277, 51279, 51301]"
5,20008094,"[50813, 50868, 50882, 50893, 50902, 50912, 50920, 50931, 50960, 50970, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51244, 51248, 51249, 51250, 51254, 51256, 51265, 51277, ..."
6,20008386,"[50868, 50882, 50893, 50902, 50912, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51248, 51249, 51250, 51265, 51275, 51277, 51279, 51301, 51678, 52172]"
7,20010230,"[50868, 50882, 50893, 50902, 50912, 50931, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275, 51277, 51279, 51301]"
8,20010239,"[50868, 50882, 50893, 50902, 50912, 50931, 50934, 50947, 50960, 50970, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51244, 51248, 51249, 51250, 51254, 51256, 51265, 51277, ..."
9,20010951,"[50868, 50882, 50893, 50902, 50912, 50920, 50931, 50960, 50970, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51244, 51248, 51249, 51250, 51254, 51256, 51265, 51277, 51279, ..."


Đã lưu danh sách Top Labs: ../data/proc/top_lab_items.csv


## 4) Ghi chú Radiology trong 12 giờ đầu (lọc theo COHORT, đọc theo `chunksize`)

**Mục đích**  
- Lấy **ghi chú radiology sớm nhất** trong **0–12 giờ** kể từ `admittime` cho mỗi `hadm_id` trong **COHORT_HADM**, làm đầu vào mô hình text→label.

**Dữ liệu đầu vào & ràng buộc**  
- `radiology.csv.gz`: `hadm_id`, `charttime`, `text`  
- `admissions_idx` (Bước 1): tra cứu `admittime` theo `hadm_id`  
- **Chỉ lấy bản ghi có `hadm_id ∈ COHORT_HADM`**. Đọc theo `chunksize` để tiết kiệm RAM.

**Xử lý**  
1) Đọc radiology theo `chunksize`, lọc theo `COHORT_HADM`.  
2) Join `admittime` từ `admissions_idx`, tính \(\Delta t\) (giờ).  
3) Giữ các ghi chú có \(0 \le \Delta t \le 12\).  
4) `normalize_text` nội dung và **chọn ghi chú sớm nhất** (nhỏ nhất `charttime`) cho mỗi `hadm_id`.

**Kết quả mong đợi**  
- `text_df`: DataFrame gồm `hadm_id`, `text` (radiology note sớm nhất trong 12h, đã chuẩn hoá).

**Ý nghĩa trường trong kết quả**  
- `hadm_id`: mã lần nhập viện (khóa chính để join).  
- `text`: văn bản ghi chú radiology đầu tiên trong 12h đầu, đã làm sạch để sẵn sàng tokenize.

In [17]:
# --- Bước 5: Lấy ghi chú xuất viện (Discharge Summary) ---
import pandas as pd

dis_path = NOTE_DIR / "discharge.csv.gz"

rows = []
for chunk in read_csv_chunks(
    dis_path,
    usecols=["hadm_id", "charttime", "text"],
    chunksize=CHUNKSIZE_DEFAULT,
):
    # Giữ lại bản ghi hợp lệ
    chunk = chunk.dropna(subset=["hadm_id", "charttime", "text"]).copy()
    if chunk.empty:
        continue

    chunk["hadm_id"] = chunk["hadm_id"].astype(int)

    # Chỉ giữ những ca có trong cohort (có note xuất viện)
    chunk = chunk[chunk["hadm_id"].isin(COHORT_HADM)]
    if chunk.empty:
        continue

    # Chuẩn hóa văn bản
    chunk["charttime"] = pd.to_datetime(chunk["charttime"], errors="coerce")
    chunk["text"] = chunk["text"].map(normalize_text)

    rows.append(chunk[["hadm_id", "charttime", "text"]])

# Gộp toàn bộ
if rows:
    dis_all = pd.concat(rows, ignore_index=True)
    # Chọn bản ghi discharge mới nhất cho mỗi hadm_id
    dis_all = dis_all.sort_values(["hadm_id", "charttime"], ascending=[True, False])
    text_df = (
        dis_all.groupby("hadm_id", as_index=False)["text"]
        .first()
        .reset_index(drop=True)
    )
else:
    text_df = pd.DataFrame(columns=["hadm_id", "text"])

# Báo cáo kết quả
print("Discharge — số hadm có ghi chú xuất viện:", text_df["hadm_id"].nunique(), "/", len(COHORT_HADM))
display(text_df.head())

print("Giải thích kết quả:")
print("- hadm_id: mã lần nhập viện (trùng khóa chuẩn).")
print("- text: nội dung ghi chú xuất viện mới nhất của mỗi ca nhập viện, đã chuẩn hóa.")


Discharge — số hadm có ghi chú xuất viện: 10000 / 10000


,hadm_id,text
0,20000815,Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: F Service: MEDICINE Allergies: walnuts Attending: ___. Chief Complaint: vomiting Major Su...
1,20004116,Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: F Service: PLASTIC Allergies: No Known Allergies / Adverse Drug Reactions Attending: ___....
2,20004829,Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: M Service: MEDICINE Allergies: Allopurinol Attending: ___ ___ Complaint: dysphagia Major ...
3,20004866,Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: F Service: MEDICINE Allergies: Codeine / Ceftin / Penicillins / Quinine / Demerol / Bactr...
4,20006409,Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: M Service: MEDICINE Allergies: Dilantin / Sulfa (Sulfonamide Antibiotics) / morphine / ab...


Giải thích kết quả:
- hadm_id: mã lần nhập viện (trùng khóa chuẩn).
- text: nội dung ghi chú xuất viện mới nhất của mỗi ca nhập viện, đã chuẩn hóa.


## 5) Ghép dữ liệu & sinh nhãn đa nhãn (multi-hot) — **theo COHORT**

**Mục đích**  
- Gom các phần: **ICD-block** (Bước 2), **lab sớm 0–6h** (Bước 3), **radiology 0–12h** (Bước 4), **demographics** (Bước 2.5) thành **một bảng mẫu/nhãn** dùng cho huấn luyện.

**Khóa nối & đầu vào**  
- Nối theo `hadm_id` giữa `icd_df`, `lab_items_by_hadm`, `text_df`.  
- Thêm demographics từ `demo_df` theo cặp khóa `[hadm_id, subject_id]`.

**Xử lý**  
1) Chuẩn hóa các cột list/text/demographics.  
2) Tạo **vocabulary**:  
   • `icd_vocab`: danh sách ICD-block phổ biến (giới hạn bởi `ICD_MAX`).  
   • `lab_vocab_items`: chính là Top-N ở Bước 3 (giữ nguyên thứ tự).  
3) Sinh vector **multi-hot** cho mỗi hàng: `y_icd`, `y_lab`.

**Kết quả mong đợi**  
- DataFrame `df_out` gồm: `hadm_id`, `subject_id`, `gender`, `age_at_admit`, `text`, `icd_blocks`, `lab_items`, `y_icd`, `y_lab`.  
- Ghi file:  
  • `examples.parquet`  
  • `vocab_meta.json` (lưu vocab & mapping `itemid→label`).

**Ý nghĩa các trường chính**  
- `icd_blocks`: danh sách ICD-block (3 ký tự).  
- `lab_items`: danh sách *duy nhất* các `itemid` thuộc vocab xuất hiện trong 0–6h.  
- `y_icd`/`y_lab`: vector multi-hot tương ứng với `icd_vocab`/`lab_vocab_items`.  
- `gender` (`M/F/U`), `age_at_admit` (0–120, `Int64`).

In [25]:
# --- Bước 5: Merge & sinh nhãn multi-hot (gồm ICD + Lab + Procedure) ---
import numpy as np
import pandas as pd
from collections import Counter
import json

# 5.1 Ghép theo hadm_id
df = (
    icd_df
    .merge(proc_df,             on="hadm_id", how="left")   # ⬅️ thêm thủ thuật
    .merge(lab_items_by_hadm,  on="hadm_id", how="left")
    .merge(text_df,             on="hadm_id", how="left")
    .merge(demo_df,             on="hadm_id", how="left")   # ⬅️ sửa ở đây
)

# 5.2 Chuẩn hoá các cột
df["icd_blocks"] = df["icd_blocks"].apply(lambda x: x if isinstance(x, list) else [])
df["proc_blocks"] = df["proc_blocks"].apply(lambda x: x if isinstance(x, list) else [])
df["lab_items"]  = df["lab_items"].apply(lambda x: x if isinstance(x, list) else [])
df["text"]       = df["text"].fillna("")
df["gender"]     = df["gender"].fillna("U").astype(str).str.upper().str[0]
df["age_at_admit"] = df["age_at_admit"].astype("Int64")

# Chỉ giữ lần nhập viện có note xuất viện
df["text"] = df["text"].str.strip()
df = df[df["text"].str.len() > 0].copy()

# 5.3 Vocabulary
cnt_icd = Counter(b for blocks in df["icd_blocks"] for b in blocks)
icd_vocab = [b for b, _ in cnt_icd.most_common(TOP_ICD)]
icd_index = {b: i for i, b in enumerate(icd_vocab)}

cnt_proc = Counter(p for ps in df["proc_blocks"] for p in ps if pd.notna(p))
proc_vocab = [p for p, _ in cnt_proc.most_common(TOP_ICD)]
proc_index = {p: i for i, p in enumerate(proc_vocab)}

lab_vocab_items = lab_vocab_df["itemid"].astype(int).tolist()
lab_index = {it: i for i, it in enumerate(lab_vocab_items)}

# 5.4 Multi-hot
def to_multihot_generic(labels, index_map, length):
    arr = np.zeros(length, dtype=np.int8)
    for t in labels:
        if t in index_map:
            arr[index_map[t]] = 1
    return arr

df["y_icd"]  = df["icd_blocks"].apply(lambda xs: to_multihot_generic(xs, icd_index, len(icd_vocab)))
df["y_lab"]  = df["lab_items"].apply(lambda xs: to_multihot_generic(xs, lab_index, len(lab_vocab_items)))
df["y_proc"] = df["proc_blocks"].apply(lambda xs: to_multihot_generic(xs, proc_index, len(proc_vocab)))

# 5.5 Ánh xạ itemid -> label (lab)
itemid_to_label = dict(zip(lab_vocab_df["itemid"].astype(int), lab_vocab_df["label"].astype(str)))

print("Kích thước vocab ICD:", len(icd_vocab))
print("Kích thước vocab Proc:", len(proc_vocab))
print("Kích thước vocab Lab (Top-N):", len(lab_vocab_items))
display(df.head())

# 5.6 Lưu ra đĩa
PROC_DIR.mkdir(parents=True, exist_ok=True)

out_examples = PROC_DIR / "examples.parquet"
df_out = df[[
    "hadm_id", "subject_id", "gender", "age_at_admit",
    "text", "icd_blocks", "proc_blocks", "lab_items",
    "y_icd", "y_proc", "y_lab"
]].copy()
df_out.to_parquet(out_examples, index=False)

out_vocab = PROC_DIR / "vocab_meta.json"
with open(out_vocab, "w") as f:
    json.dump({
        "icd_vocab": icd_vocab,
        "n_icd": len(icd_vocab),
        "proc_vocab": proc_vocab,
        "n_proc": len(proc_vocab),
        "lab_vocab_items": lab_vocab_items,
        "n_lab": len(lab_vocab_items),
        "itemid_to_label": {int(k): str(v) for k, v in itemid_to_label.items()}
    }, f, indent=2)

print("Đã lưu (chỉ các HADM có note):")
print("-", out_examples)
print("-", out_vocab)


Kích thước vocab ICD: 50
Kích thước vocab Proc: 50
Kích thước vocab Lab (Top-N): 50


,subject_id_x,hadm_id,icd_blocks,subject_id_y,proc_blocks,lab_items,text,subject_id,gender,age_at_admit,y_icd,y_lab,y_proc
0,10000935,21738619,"[266, 272, 276, 288, 311, 427, 715, 721, 780, 786, 787, 793, V15, V45, V58]",NaN,[],"[50813, 50861, 50868, 50878, 50882, 50885, 50902, 50912, 50931, 50960, 50971, 50983, 51006, 51221, 51222, 51237, 51248, 51249, 51250, 51265, 51274, 51275, 51277, 51279, 51301]",Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: F Service: MEDICINE Allergies: Sulfa (Sulfonamide Antibiotics) / Codeine / Bactrim Attend...,10000935,F,57,"[0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,10000935,25849114,"[153, 197, 266, 272, 276, 285, 286, 288, 300, 311, 348, 427, 578, 715, 721, 729, 788, V15, V16, V45, V49, V88]",10000935.0,[501],"[50802, 50804, 50813, 50818, 50820, 50821, 50861, 50862, 50863, 50868, 50878, 50882, 50885, 50893, 50902, 50912, 50920, 50931, 50954, 50960, 50970, 50971, 50983, 51006, 51146, ...",Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: F Service: MEDICINE Allergies: Sulfa (Sulfonamide Antibiotics) / Codeine / Bactrim Attend...,10000935,F,57,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,10000935,26381316,"[197, 199, 266, 272, 276, 288, 311, 338, 564, 715, 721, 724, 780, 782, 783, 786, 787, 789, V10, V15, V45, V85]",NaN,[],"[50861, 50863, 50868, 50878, 50882, 50885, 50893, 50902, 50912, 50931, 50954, 50960, 50970, 50971, 50983, 51006, 51221, 51222, 51248, 51249, 51250, 51265, 51277, 51279, 51301]",Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: F Service: MEDICINE Allergies: Sulfa (Sulfonamide Antibiotics) / Codeine / Bactrim Attend...,10000935,F,57,"[0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,10000935,29541074,"[266, 272, 278, 311, 560, 788, 998, E87, V10, V88]",10000935.0,"[456, 545]","[50868, 50882, 50893, 50902, 50912, 50920, 50931, 50960, 50970, 50971, 50983, 51006, 51146, 51200, 51221, 51222, 51244, 51248, 51249, 51250, 51254, 51256, 51265, 51277, 51279, ...",Name: ___ Unit No: ___ Admission Date: ___ Discharge Date: ___ Date of Birth: ___ Sex: F Service: SURGERY Allergies: Sulfonamides / Codeine / Bactrim Attending: ___. Chief Comp...,10000935,F,53,"[0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]"
4,10002430,24513842,"[E78, E87, I10, I25, I27, I28, I

Đã lưu (chỉ các HADM có note):
- ../data/proc/examples.parquet
- ../data/proc/vocab_meta.json


In [ ]:
# Kiểm tra 5 mẫu có lab_items KHÔNG rỗng (và y_lab có ít nhất 1 nhãn)
import numpy as np
import pandas as pd

def has_lab_items(x):
    return isinstance(x, (list, tuple)) and len(x) > 0

def has_y_lab(x):
    try:
        return np.asarray(x).sum() > 0
    except Exception:
        return False

# Nếu bạn đã lưu ra df_out (ở Bước 5), thì đọc lại; còn nếu đang có biến df thì dùng df
try:
    _ = df
    frame = df
except NameError:
    frame = pd.read_parquet(PROC_DIR / "examples.parquet")

mask_items = frame["lab_items"].apply(has_lab_items)
mask_y     = frame["y_lab"].apply(has_y_lab)

nonempty_lab = frame[mask_items].copy()
print(f"Số ca có lab_items không rỗng: {len(nonempty_lab)} / {len(frame)} "
      f"({len(nonempty_lab)/len(frame):.1%})")

nonempty_both = frame[mask_items & mask_y].copy()
print(f"Số ca có lab_items≠∅ và y_lab>0: {len(nonempty_both)}")

# Lấy ngẫu nhiên 5 mẫu để xem chi tiết
sample_n = min(5, len(nonempty_lab))
sample_rows = nonempty_lab.sample(sample_n, random_state=42) if sample_n > 0 else nonempty_lab.head(0)

cols_show = ["subject_id", "hadm_id", "gender", "age_at_admit", "icd_blocks", "lab_items", "text"]
display(sample_rows[cols_show])

# (Tuỳ chọn) In kèm tổng số nhãn lab được bật để xác nhận khớp y_lab
if sample_n > 0:
    tmp = sample_rows.copy()
    tmp["y_lab_sum"] = tmp["y_lab"].apply(lambda a: int(np.asarray(a).sum()) if isinstance(a, (list,np.ndarray)) else 0)
    display(tmp[["hadm_id", "lab_items", "y_lab_sum"]])
